In [1]:
import os

os.chdir(r'C:\Users\carlo\GitHub\ProjectBuffalo')

import buffalo.ingestion as ingestion
import buffalo.predictor as predictor
import buffalo.algorithm as algorithm
import buffalo.predictor.models as modeling
import torch
import pickle
import pandas as pd
from buffalo.utility import expand_grid
from tqdm.auto import tqdm

torch.manual_seed(0)

In [ ]:
ingestor = ingestion.DataIngestion(ingestion.enum.API.ADVANTAGE)

In [ ]:
ingestor.load_data(r'cached_data/ingestion.sqlite')

In [ ]:
target_stock = ingestor.data['ADJUSTED_DAILY_STOCK']

In [ ]:
target_income_statement = ingestor.data['COMPANY_INCOME_STATEMENT'].query('freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function']).dropna(axis=1, how='all')
target_balance_sheet = ingestor.data['COMPANY_BALANCE_SHEET'].query('freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function']).dropna(axis=1, how='all')
target_cash_flow = ingestor.data['COMPANY_CASH_FLOW'].query('freq == "quarterly"').drop(columns=['reported_currency', 'symbol', 'freq', 'function', 'net_income']).dropna(axis=1, how='all')

In [ ]:
fed_funds_rate = ingestor.data['FEDERAL_FUNDS_RATE'][['value']].rename(columns={'value': 'effective_federal_funds_rate'}).dropna(axis=1, how='all')
payroll = ingestor.data['NONFARM_PAYROLL'][['value']].rename(columns={'value': 'total_nonfarm_payroll'}).dropna(axis=1, how='all')
cpi = ingestor.data['CPI'][['value']].rename(columns={'value': 'consumer_price_index'}).dropna(axis=1, how='all')
unemployment = ingestor.data['UNEMPLOYMENT'][['value']].rename(columns={'value': 'unemployment_rate'}).dropna(axis=1, how='all')
real_gdp = ingestor.data['REAL_GDP'][['value']].rename(columns={'value': 'real_gross_domestic_product'}).dropna(axis=1, how='all')
real_gdp_per_capita = ingestor.data['REAL_GDP_PER_CAPITA'][['value']].rename(columns={'value': 'real_gross_domestic_product_per_capita'})
treasury_yield = ingestor.data['TREASURY_YIELD'][['value', 'maturity']].pivot(columns=['maturity'], values=['value']).dropna(axis=1, how='all')
treasury_yield.columns = 'treasury_yield_' + treasury_yield.columns.droplevel(level=0)

In [ ]:
sma = ingestor.data['SMA']
sma = sma[sma['interval'] == "daily"]
roc = ingestor.data['ROC']
roc = roc[roc['interval'] == "daily"]
ht_sine = ingestor.data['HT_SINE']
ht_sine = ht_sine[ht_sine['interval'] == "daily"]
mom = ingestor.data['MOM']
mom = mom[mom['interval'] == "daily"]

In [ ]:
n_ahead = 1

In [ ]:
## For each symbol, build a dataset
dataset = []
for sym, stock_df in tqdm(target_stock.groupby('symbol')):
    stock_df = stock_df.drop(columns=['close', 'dividend_amount', 'split_coefficient', 'symbol', 'interval'])
    stock_df = predictor.util.align_dataframe_by_time(stock_df, fed_funds_rate)
    stock_df = predictor.util.align_dataframe_by_time(stock_df, payroll)
    stock_df = predictor.util.align_dataframe_by_time(stock_df, cpi)
    stock_df = predictor.util.align_dataframe_by_time(stock_df, unemployment)
    stock_df = predictor.util.align_dataframe_by_time(stock_df, real_gdp)
    stock_df = predictor.util.align_dataframe_by_time(stock_df, real_gdp_per_capita)
    stock_df = predictor.util.align_dataframe_by_time(stock_df, treasury_yield)
    curr_sma = sma.query(f'symbol == "{sym}"').drop(columns=['symbol', 'function', 'interval'])
    curr_sma = curr_sma.pivot(columns=['time_period', 'series_type'], values=['sma']).dropna(axis=1, how='all')
    curr_sma.columns = curr_sma.columns.map(lambda x: '-'.join([str(t) for t in x]))
    stock_df = predictor.util.align_dataframe_by_time(stock_df, curr_sma)
    curr_roc = roc.query(f'symbol == "{sym}"').drop(columns=['symbol', 'function', 'interval'])
    curr_roc = curr_roc.pivot(columns=['time_period', 'series_type'], values=['roc']).dropna(axis=1, how='all')
    curr_roc.columns = curr_roc.columns.map(lambda x: '-'.join([str(t) for t in x]))
    stock_df = predictor.util.align_dataframe_by_time(stock_df, curr_roc)
    curr_ht_sine = ht_sine.query(f'symbol == "{sym}"').drop(columns=['symbol', 'function', 'interval'])
    curr_ht_sine = curr_ht_sine.pivot(columns=['time_period', 'series_type'], values=['lead_sine', 'sine']).dropna(axis=1, how='all')
    curr_ht_sine.columns = curr_ht_sine.columns.map(lambda x: '-'.join([str(t) for t in x]))
    stock_df = predictor.util.align_dataframe_by_time(stock_df, curr_ht_sine)
    curr_mom = mom.query(f'symbol == "{sym}"').drop(columns=['symbol', 'function', 'interval'])
    curr_mom = curr_mom.pivot(columns=['time_period', 'series_type'], values=['mom']).dropna(axis=1, how='all')
    curr_mom.columns = curr_mom.columns.map(lambda x: '-'.join([str(t) for t in x]))
    stock_df = predictor.util.align_dataframe_by_time(stock_df, curr_mom)
    time_series_data = predictor.util.TimeSeriesData(endog=stock_df[['adjusted_close']], exog=stock_df.drop(columns=['adjusted_close']), seq_len=180, label_len=n_ahead, n_ahead=n_ahead, name=f'DAILY_ADJUSTED_CLOSE_{sym}')
    dataset.append(time_series_data)

In [ ]:
dataset = dataset[1:]

In [ ]:
pickle.dump(dataset, open(r'cached_data/target_stock.pickle', 'wb'))

In [ ]:
dataset = pickle.load(open(r'cached_data/target_stock.pickle', 'rb'))

In [ ]:
time_series_data = predictor.util.TimeSeriesDataCollection(dataset)

#### Offline Learning

In [ ]:
sweep_params = expand_grid(
    batch_size=64,
    n_fold=1,
    learning_rate=[0.0001, 0.001],
    weight_decay=[0.001, 0.0001],
    dropout=[0.2, 0.4],
    epochs=[120],
    bidirectional = [True, False],
    hidden_size=[32, 64, 128],
    num_layers=[4, 8]
)

In [ ]:
for i in tqdm(range(sweep_params.shape[0])):
    param = sweep_params.loc[i,:].to_dict()
    rnn = modeling.RNN(
        input_size=77,
        n_ahead=n_ahead,
        hidden_size=param['hidden_size'],
        output_size=1,
        num_layers=param['num_layers'],
        dropout=param['dropout'],
        bidirectional=param['bidirectional'],
        use_gpu=True)
    optimizer = torch.optim.Adam(
        rnn.parameters(),
        lr=param['learning_rate'],
        weight_decay=param['weight_decay'])
    loss_func = torch.nn.MSELoss()
    training_record = predictor.train_and_evaluate_model(
        rnn,
        optimizer,
        loss_func,
        time_series_data,
        epochs=param['epochs'],
        test_ratio=0.2,
        n_fold=param['n_fold'],
        clip_grad=1,
        batch_size=param['batch_size']) # Pointwise prediction
    training_record.serialize_to_file(r'cached_data/massmodel_record.sqlite', additional_note_dataset='', additonal_note_model='')

In [ ]:
for i in tqdm(range(sweep_params.shape[0])):
    param = sweep_params.loc[i,:].to_dict()
    rnn = modeling.LSTM(
        input_size=77,
        n_ahead=n_ahead,
        hidden_size=param['hidden_size'],
        output_size=1,
        num_layers=param['num_layers'],
        dropout=param['dropout'],
        bidirectional=param['bidirectional'],
        use_gpu=True)
    optimizer = torch.optim.Adam(
        rnn.parameters(),
        lr=param['learning_rate'],
        weight_decay=param['weight_decay'])
    loss_func = torch.nn.MSELoss()
    training_record = predictor.train_and_evaluate_model(
        rnn,
        optimizer,
        loss_func,
        time_series_data,
        epochs=param['epochs'],
        test_ratio=0.2,
        n_fold=param['n_fold'],
        clip_grad=1,
        batch_size=param['batch_size']) # Pointwise prediction
    training_record.serialize_to_file(r'cached_data/massmodel_record.sqlite', additional_note_dataset='', additonal_note_model='')

#### Online Learning

In [ ]:
sweep_params = expand_grid(
    hidden_size=[32, 64, 128],
    num_layers=[1, 2, 3, 4],
    dropout=[0.0, 0.2, 0.4],
    batch_size=[32, 64, 128],
    learning_rate=[0.001, 0.005, 0.0001],
    weight_decay=[0.001, 0.0001, 0.00001],
    epochs=[40],
    epochs_per_update=[1, 5, 10, 15],
    update_freq=[1, 5, 10, 15],
    bidirectional = [True, False]
)

In [ ]:
for i in tqdm(range(sweep_params.shape[0])):
    rnn = modeling.RNN(
        input_size=target_stock.shape[1],
        n_ahead=n_ahead,
        hidden_size=sweep_params.loc[i,'hidden_size'],
        output_size=time_series_data.endog.shape[1],
        num_layers=sweep_params.loc[i,'num_layers'],
        dropout=sweep_params.loc[i,'dropout'],
        bidirectional=sweep_params.loc[i,'bidirectional'],
        use_gpu=True)
    optimizer = torch.optim.Adam(
        rnn.parameters(),
        lr=sweep_params.loc[i,'learning_rate'],
        weight_decay=sweep_params.loc[i,'weight_decay'])
    loss_func = torch.nn.MSELoss()
    update_rule = algorithm.online_update.IncrementalBatchGradientDescent(epochs=sweep_params.loc[i,'epochs'], epochs_per_update=sweep_params.loc[i,'epochs_per_update'], update_freq=sweep_params.loc[i,'update_freq'], clip_grad_norm_update=None, clip_grad_norm_train=1)
    training_record = predictor.train_and_evaluate_model_online(
        rnn,
        time_series_data,
        update_rule,
        optimizer,
        loss_func,
        train_ratio=0.3,
        batch_size=sweep_params.loc[i,'batch_size']) # Pointwise prediction
    training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], n_ahead=1, hidden_size=64, output_size=1, num_layers=2, dropout=0.5, bidirectional=False, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.01)
update_rule = algorithm.online_update.IncrementalBatchGradientDescent(epochs=80, epochs_per_update=10, update_freq=5, clip_grad_norm_update=None, clip_grad_norm_train=1)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model_online(rnn,
                                                            time_series_data,
                                                            update_rule,
                                                            optimizer,
                                                            loss_func,
                                                            train_ratio=0.3,
                                                            batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')

In [ ]:
training_record = predictor.util.ModelPerformanceOnline.deserialize_from_file(r'cached_data/record.sqlite', 2)
training_record.plot_training_records()
training_record.plot_logs()
training_record.plot_residuals()

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], n_ahead=1, hidden_size=64, output_size=1, num_layers=2, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
update_rule = algorithm.online_update.IncrementalBatchGradientDescent(epochs=80, epochs_per_update=5, update_freq=5, clip_grad_norm_update=None, clip_grad_norm_train=1)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model_online(rnn,
                                                            time_series_data,
                                                            update_rule,
                                                            optimizer,
                                                            loss_func,
                                                            train_ratio=0.3,
                                                            batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record = predictor.util.ModelPerformanceOnline.deserialize_from_file(r'cached_data/record.sqlite', 1)
training_record.plot_training_records()
training_record.plot_logs()
training_record.plot_residuals()

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], n_ahead=1, hidden_size=64, output_size=1, num_layers=3, dropout=0.2, bidirectional=False, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
update_rule = algorithm.online_update.IncrementalBatchGradientDescent(epochs=80, epochs_per_update=5, update_freq=5, clip_grad_norm_update=None, clip_grad_norm_train=1)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model_online(rnn,
                                                            time_series_data,
                                                            update_rule,
                                                            optimizer,
                                                            loss_func,
                                                            train_ratio=0.3,
                                                            batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record = predictor.util.ModelPerformanceOnline.deserialize_from_file(r'cached_data/record.sqlite', 1)
training_record.plot_training_records()
training_record.plot_logs()
training_record.plot_residuals()

In [ ]:
rnn = modeling.RNN(input_size=target_stock.shape[1], n_ahead=1, hidden_size=64, output_size=1, num_layers=3, dropout=0.2, bidirectional=True, use_gpu=True)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001, weight_decay=0.001)
update_rule = algorithm.online_update.IncrementalBatchGradientDescent(epochs=80, epochs_per_update=5, update_freq=5, clip_grad_norm_update=None, clip_grad_norm_train=1)
loss_func = torch.nn.MSELoss()
training_record = predictor.train_and_evaluate_model_online(rnn,
                                                            time_series_data,
                                                            update_rule,
                                                            optimizer,
                                                            loss_func,
                                                            train_ratio=0.3,
                                                            batch_size=64) # Pointwise prediction
training_record.serialize_to_file(r'cached_data/record.sqlite', additional_note_dataset='', additonal_note_model='')
training_record = predictor.util.ModelPerformanceOnline.deserialize_from_file(r'cached_data/record.sqlite', 1)
training_record.plot_training_records()
training_record.plot_logs()
training_record.plot_residuals()